In [32]:
# Импорт необходимых библиотек
import pandas as pd
import sqlite3
from datetime import datetime, timedelta

In [4]:
# Создадим базу данных с которой будем работать
CONN = sqlite3.connect('sqlite3.db')

In [9]:
# Напишите функцию которая скачивает курс валюты EUR/USD с гитхаба
# Функция принимает на вход дату
# Функция должна вернуть список словарей или можно использовать можно использоват pandas.read_csv()

# ссылка на гитхаб с файлами - https://github.com/datanlnja/airflow_course/tree/main/excangerate

def extract_currency(date):
    currency = pd.read_csv(f'https://raw.githubusercontent.com/datanlnja/airflow_course/main/excangerate/{date}.csv')
    return currency

In [10]:
#проверка
extract_currency('2021-01-02')

,date,currency_from,currency_to,amount
0,2021-01-02,eur,usd,1.22


In [19]:
# Напишите функцию которая скачивает данные с гитхаба
# Функция принимает на вход дату
# Функция должна вернуть список словарей или можно использовать можно использоват pandas.read_csv()

# ссылка на гитхаб с файлами - https://github.com/datanlnja/airflow_course/tree/main/data

def extract_data(date):
    data = pd.read_csv(f'https://raw.githubusercontent.com/datanlnja/airflow_course/main/data/{date}.csv')
    return data

In [20]:
#проверка
extract_data('2021-01-02')

,currency,value,date
0,EUR,27,2021-01-02
1,EUR,71,2021-01-02
2,EUR,39,2021-01-02
3,EUR,94,2021-01-02
4,EUR,45,2021-01-02
5,EUR,3,2021-01-02
6,EUR,49,2021-01-02
7,EUR,96,2021-01-02
8,EUR,36,2021-01-02


In [28]:
# Напишите функцию которая принимает на вход список словарей, или пандас датафрейм
# и загружает в табличку в sqlite, можно использоват pandas.to_sql()
def insert_to_db(data, table_name, conn):
    data.to_sql( name= table_name, con = conn, if_exists='append')

In [44]:
#проверка
data = extract_data('2021-01-02')
insert_to_db(data, 'data', CONN)

In [45]:
# Напишите функцию которая выполнит SQL запрос к базу данных
# Здесь удобно использовать sqlalchemy или https://pythonru.com/biblioteki/vvedenie-v-sqlite-python
# Если выполняется select ... то функция должна возвращать результат (не None)
# Если используется внутренняя операция например truncate table, insert into select ... то возвращает None
# data = sql_query(sql="select * from table") -> [(1, 2, 3), (4, 5, 6) ...]
# sql_query("truncate table") -> None

def sql_query(sql, conn):
    cursor = CONN.cursor()
    cursor.execute(sql)
    record = cursor.fetchall()
    cursor.close()
    if sql.split()[0].lower()=='select':
        return record

In [46]:
#проверка
sql_query('select * from data', CONN)

[(0, 'EUR', 27, '2021-01-02'),
 (1, 'EUR', 71, '2021-01-02'),
 (2, 'EUR', 39, '2021-01-02'),
 (3, 'EUR', 94, '2021-01-02'),
 (4, 'EUR', 45, '2021-01-02'),
 (5, 'EUR', 3, '2021-01-02'),
 (6, 'EUR', 49, '2021-01-02'),
 (7, 'EUR', 96, '2021-01-02'),
 (8, 'EUR', 36, '2021-01-02')]

In [47]:
sql_query('drop table data', CONN)

In [38]:
# Напишите генерацию дат, так чтобы у вас получился список
# 2021-01-02, 2021-01-03 ... etc
# Нужны даты с 2021-01-01 по 2021-01-04
start_date = datetime(2021, 1, 1)
end_date = datetime(2021, 1, 5)

dates_list = []
current_date = start_date

while current_date < end_date:
    dates_list.append(current_date.strftime("%Y-%m-%d"))
    current_date += timedelta(days=1)

In [79]:
# Запустите ваш код в функции main   
def main(date, conn):

  # Выгружаем данные по валютам и из источника
  currency = extract_currency(date)
  data = extract_data(date)

  # Создайте необходимые таблицы если это нужно
  # 'currency', 'data' и 'join_data (таблица с объединенными данными)

  sql_query('create table if not exists join_data(date DateOnly,code TEXT,base TEXT,rate REAL ,value INTEGER)', conn)

  # Вставляем данные в БД
  # У вас долнжо получиться 2 таблицы CURRENCY, DATA
  insert_to_db(currency, 'currency', conn)
  insert_to_db(data, 'data',conn)

  # Объединение данных в отдельную таблицу JOIN_DATA
  # CURRENCY, DATA объединить через JOIN по дате и валюте
  # Используйте insert into select ... inner join ...
  # И соответственно создать таблицу для этого
    
  sql_query('insert into JOIN_DATA(date,code,base,rate,value) select distinct c.date, upper(currency_to), d.currency, amount, value from currency c join data d on d.date = c.date and d.currency = upper(c.currency_from)', conn)

  # Очистка временных таблиц CURRENCY, DATA
  # Это те таблицы куда произошла вставка данных
  # Используйте truncate
    
  sql_query('delete from data', conn) #кажется у sqlite нет truncate
  sql_query('delete from currency',conn)


# Пройдемся по списку дат и выполним скрипт
for date in dates_list:
    main(date, CONN)

In [80]:
# чтобы првоерить решение можете обратиться к вашей базе данных таким образом

import sqlite3

# Создаем подключение к базе данных
conn = sqlite3.connect('sqlite3.db')

# Создаем курсор для выполнения SQL-запросов
cursor = conn.cursor()

# Выполняем запрос к таблице
cursor.execute("SELECT * FROM JOIN_DATA")

# Извлекаем все строки из результата запроса
rows = cursor.fetchall()

# Выводим результаты
for row in rows:
    print(row)

# Закрываем соединение
conn.close()

('2021-01-01', 'USD', 'EUR', 1.21, 23)
('2021-01-01', 'USD', 'EUR', 1.21, 24)
('2021-01-01', 'USD', 'EUR', 1.21, 38)
('2021-01-01', 'USD', 'EUR', 1.21, 42)
('2021-01-01', 'USD', 'EUR', 1.21, 48)
('2021-01-01', 'USD', 'EUR', 1.21, 65)
('2021-01-01', 'USD', 'EUR', 1.21, 74)
('2021-01-01', 'USD', 'EUR', 1.21, 86)
('2021-01-02', 'USD', 'EUR', 1.22, 3)
('2021-01-02', 'USD', 'EUR', 1.22, 27)
('2021-01-02', 'USD', 'EUR', 1.22, 36)
('2021-01-02', 'USD', 'EUR', 1.22, 39)
('2021-01-02', 'USD', 'EUR', 1.22, 45)
('2021-01-02', 'USD', 'EUR', 1.22, 49)
('2021-01-02', 'USD', 'EUR', 1.22, 71)
('2021-01-02', 'USD', 'EUR', 1.22, 94)
('2021-01-02', 'USD', 'EUR', 1.22, 96)
('2021-01-03', 'USD', 'EUR', 1.22, 2)
('2021-01-03', 'USD', 'EUR', 1.22, 23)
('2021-01-03', 'USD', 'EUR', 1.22, 35)
('2021-01-03', 'USD', 'EUR', 1.22, 40)
('2021-01-03', 'USD', 'EUR', 1.22, 41)
('2021-01-03', 'USD', 'EUR', 1.22, 56)
('2021-01-03', 'USD', 'EUR', 1.22, 93)
('2021-01-03', 'USD', 'EUR', 1.22, 100)
